In [34]:
# import modules from root of project
%cd ..

from arena.arena import parse_stimulus_matrix, Sheet, run_sheets, collect_actuation_sheets, SheetInvocation, \
    lql_to_sheet_signature
from arena.engine.adaptation import PassThroughAdaptationStrategy
from arena.engine.artifacts import CodeCandidate, import_classes_under_test
from arena.engine.classes import ClassUnderTest
from arena.engine.ssntestdriver import interpret_sheet, run_sheet, InvocationListener, Test, TestInvocation
from arena.lql.lqlparser import parse_lql
from arena.ssn.ssnparser import parse_sheet

# pandas
import pandas as pd
pd.set_option('display.max_columns', 0) #replace n with the number of columns you want to see completely
pd.set_option('display.max_rows', 0) #replace n with the number of rows you want to see completely

/


/home/marcus/PycharmProjects/arena-python/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Create Stimulus Matrix for List Example

In [35]:
# lql (interface specification)
lql = """List {
    append(object)->void
    pop()->object
    len()->int }
"""

# stimulus sheet
ssn_jsonl = """
            {"cells": {"A1": {}, "B1": "create", "C1": "List"}}
            {"cells": {"A2": {}, "B2": "append", "C2": "A1", "D2": "'Hello World!'"}}
            {"cells": {"A3": 1,  "B3": "len",  "C3": "A1"}}
            {"cells": {"A4": {}, "B4": "pop", "C4": "A1"}}
            {"cells": {"A5": 0,  "B5": "len", "C5": "A1"}}
"""

# classes under test
#    cuts = [ClassUnderTest("1", list), ClassUnderTest("2", collections.deque)] works as well
cuts = [ClassUnderTest("1", "list"), ClassUnderTest("2", "collections.deque")]

# create stimulus matrix
sm = parse_stimulus_matrix([Sheet("test1()", ssn_jsonl, lql)], cuts, [SheetInvocation("test1", "")])

## SM is a pandas DataFrame

The data frame consists of complex objects.

In [36]:
sm

,<arena.engine.classes.ClassUnderTest object at 0x7579b4250b30>,<arena.engine.classes.ClassUnderTest object at 0x7579cabc20c0>
0,<arena.engine.ssntestdriver.TestInvocation obj...,<arena.engine.ssntestdriver.TestInvocation obj...


## Run Stimulus Matrix in Arena

In [37]:
# run stimulus matrix
invocation_listener = InvocationListener()
srm = run_sheets(sm, 1, invocation_listener)

## SRM is a pandas DataFrame

The data frame consists of complex objects.

In [38]:
srm

,<arena.engine.adaptation.AdaptedImplementation object at 0x7579b5126210>,<arena.engine.adaptation.AdaptedImplementation object at 0x7579d067b200>
0,Executed Invocations\n0 => ExecutedInvocation ...,Executed Invocations\n0 => ExecutedInvocation ...


## Zoom into SRM

In [39]:
# create actuation sheets, now we have the real stimulus response matrix (SRM)
srm_actuations = collect_actuation_sheets(srm)

In [40]:
srm_actuations

,<arena.engine.adaptation.AdaptedImplementation object at 0x7579b5126210>,<arena.engine.adaptation.AdaptedImplementation object at 0x7579d067b200>
0,output operation service ...,output operation service ...


### Get Actuation sheet for first cell in the DataFrame

In [45]:
srm_actuations.iloc[0][0] # pick first cell

/tmp/ipykernel_271683/2780891456.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  srm_actuations.iloc[0][0] # pick first cell


,output,operation,service,input_0
0,$CUT@list@0,__init__,UORJYPAOKJ.List,None
1,None,append,$CUT@list@0,Hello World!
2,1,len,$CUT@list@0,None
3,Hello World!,pop,$CUT@list@0,None
4,0,len,$CUT@list@0,None


### Which Candidate Implementation is it?

In [46]:
vars(srm_actuations.columns[0]) # print properties of object

{'cut': <arena.engine.classes.ClassUnderTest at 0x7579b4250b30>,
 'initializer_mapping': {},
 'method_mapping': {<arena.lql.lqlparser.MethodSignature at 0x7579b425b770>: <method 'append' of 'list' objects>,
  <arena.lql.lqlparser.MethodSignature at 0x7579b425bad0>: <method 'pop' of 'list' objects>,
  <arena.lql.lqlparser.MethodSignature at 0x7579b4258bf0>: <function len(obj, /)>},
 'adapter_id': '0'}

In [47]:
#  show class
vars(srm_actuations.columns[0].cut) # print properties of object

{'id': '1', 'code_candidate': None, 'class_under_test': list}

### Which Test is it?

In [48]:
# from Stimulus Matrix
vars(sm.iloc[0][0].test) # print properties of object

/tmp/ipykernel_271683/1479926970.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  vars(sm.iloc[0][0].test) # print properties of object


{'name': 'test1',
 'parsed_sheet': <arena.ssn.ssnparser.ParsedSheet at 0x7579b42599a0>,
 'interface_specification': <arena.lql.lqlparser.Interface at 0x7579b425ba40>,
 'signature': <arena.arena.SheetSignature at 0x7579b425bfe0>}